In [1]:
import torch
import src.models as mdls
import src.utils as utils
import torchvision.transforms as tf
import numpy as np
import cv2

In [ ]:
conv = mdls.ConvEmbedding()
model = mdls.Triplet(conv)

path_model_dict = "/Users/andreisuhov/Downloads/checkpoint_epoch_19_recognition.pth"
model_state_dict = torch.load(path_model_dict, map_location=torch.device('cpu'))['model_state_dict']
model.load_state_dict(model_state_dict)
model.eval()
None

In [11]:
first = '/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.52.07.png'
second = "/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.52.17.png"
third = '/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.52.32.png'
fourth = "/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.52.24.png"

five = '/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.55.36.png'
six = '/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.56.19.png'
ant = '/Users/andreisuhov/Desktop/новые фотки/нов фото/Снимок экрана 2023-11-03 в 20.58.32.png'


def get_emb(image_path):
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128,128)).astype(np.float32)
    img /= 255.0
    img = np.transpose(img, (2, 0, 1))
    img = torch.tensor(img, dtype=torch.float32)
    
    with torch.no_grad():
        emb = model.encoder(img.unsqueeze(0))

    return emb.squeeze()

In [12]:
f_emb = get_emb(first)
s_emb = get_emb(second)
t_emb = get_emb(third)
fo_emb = get_emb(fourth)
fiv_emb = get_emb(five)
six_emb = get_emb(six)
ant_emb = get_emb(ant)
fo_emb.shape

torch.Size([512])

In [15]:
import numpy as np
from scipy.spatial.distance import cosine

# предположим, что f_emb, s_emb, t_emb и fo_emb - это ваши четыре
def euclidean_distance(tensor1, tensor2):
    return torch.dist(tensor1, tensor2)


# Инициализируем матрицу расстояний
distance_matrix = torch.zeros(7, 7)

# Заполняем матрицу расстояниями
embeddings = [f_emb, s_emb, t_emb, fo_emb, fiv_emb, six_emb, ant_emb]
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        if i != j:
            distance_matrix[i, j] = euclidean_distance(embeddings[i].squeeze(), embeddings[j].squeeze())

# Выводим матрицу расстояний
print(distance_matrix)

tensor([[0.0000, 2.5470, 4.4009, 3.4439, 2.3348, 2.9689, 3.1880],
        [2.5470, 0.0000, 3.3681, 3.5720, 2.6139, 3.4475, 3.2915],
        [4.4009, 3.3681, 0.0000, 4.3906, 4.1000, 4.2492, 4.6798],
        [3.4439, 3.5720, 4.3906, 0.0000, 2.7745, 3.2115, 2.7830],
        [2.3348, 2.6139, 4.1000, 2.7745, 0.0000, 2.3802, 2.7478],
        [2.9689, 3.4475, 4.2492, 3.2115, 2.3802, 0.0000, 3.5213],
        [3.1880, 3.2915, 4.6798, 2.7830, 2.7478, 3.5213, 0.0000]])
